# Implementation of a 3D Unet Architecture


In [4]:
import numpy as np 
import matplotlib.pyplot as plt 
import os, time
from importlib import reload

# 3D visualization tools
from mayavi import mlab
mlab.init_notebook(backend='ipy')

import tensorflow as tf
import model, utilities

Notebook initialized with ipy backend.


In [19]:
reload(utilities)
reload(model)

<module 'model' from 'c:\\Users\\Linus Meienberg\\Google Drive\\Janelia\\ImageSegmentation\\3D Unet\\model.py'>

In [3]:
# Fix for tensorflow-gpu issues that I found online... (don't ask me what it does)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
# Get permissible network input sized (cube lengths)
n_blocks = 2
valid_inputs = [n for n in range(512) if utilities.check_size(n, n_blocks=n_blocks)[0]]
print(valid_inputs)
print('Output shape at {} is {}'.format(252,utilities.check_size(252, n_blocks)[1]))

[92, 100, 108, 116, 124, 132, 140, 148, 156, 164, 172, 180, 188, 196, 204, 212, 220, 228, 236, 244, 252, 260, 268, 276, 284, 292, 300, 308, 316, 324, 332, 340, 348, 356, 364, 372, 380, 388, 396, 404, 412, 420, 428, 436, 444, 452, 460, 468, 476, 484, 492, 500, 508]
Output shape at 252 is 164.0


In [5]:
unet = model.Unet(n_blocks=2, initial_filters=8)

In [5]:
unet(tf.ones((1,252,252,252,1)))

ResourceExhaustedError: OOM when allocating tensor with shape[1,48,168,168,168] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv3D]

In [11]:
unet.summary()

Model: "Unet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_block_1 (InputBlock)   multiple                  57984     
_________________________________________________________________
downsample_block_1 (Downsamp multiple                  331968    
_________________________________________________________________
downsample_block_2 (Downsamp multiple                  1327488   
_________________________________________________________________
bottleneck_block_1 (Bottlene multiple                  7406848   
_________________________________________________________________
upsample_block_1 (UpsampleBl multiple                  7602944   
_________________________________________________________________
upsample_block_2 (UpsampleBl multiple                  1900928   
_________________________________________________________________
output_block_1 (OutputBlock) multiple                  442626 

In [30]:
sample_img = utilities.get_sample_volume()
sample_mask = utilities.get_sample_mask()

In [31]:
mlab.figure(size=(500,500))
plot = mlab.contour3d(sample_img[:,:,:,0], contours = 4, transparent=True)
plot

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xf4\x00\x00\x01\xf4\x08\x02\x00\x00\x00D\xb4H\xd…

In [33]:
mlab.figure(size=(500,500))
plot = mlab.contour3d(sample_mask[:,:,:,0])
plot 

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xf4\x00\x00\x01\xf4\x08\x02\x00\x00\x00D\xb4H\xd…

In [36]:
np.histogram(sample_mask, bins=4)
#NOTE The mask seems to contain integer values up to 3 how does that come? what does it signify? 

(array([9585602, 1061420,       0,     978], dtype=int64),
 array([0.  , 0.75, 1.5 , 2.25, 3.  ], dtype=float32))